In [1]:
import numpy as np
from keras.datasets import mnist

import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1' 
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.48
set_session(tf.Session(config=config))

import sys  
sys.path.append('../')
from util.util import *
from util import importNotebook
from module.autoencoder import VAE, ConvVAE

Using TensorFlow backend.


importing Jupyter notebook from ../module/autoencoder.ipynb


# Example of VAE

In [ ]:
# Load dataset
(xTrain, yTrain), (xTest, yTest) = mnist.load_data()
xTrain = xTrain.astype('float32') / 255.
xTest = xTest.astype('float32') / 255.
numTrain = len(xTrain)
numTest = len(xTest)
sizeDigit = xTrain.shape[1:]
dimInput = np.prod(xTrain.shape[1:]) # dimInput is width*height
xTrain = xTrain.reshape((numTrain, dimInput))
xTest = xTest.reshape((numTest, dimInput))
print(xTrain.shape)
print(xTest.shape)

# Set parameters
numEpochs = 10
sizeBatch = 128
sizeKernel = 3
layerDense = [16, 2]
layerConv = [4, 16]
ratRecon = 1
nameOptim = 'adam'
pathTempBest = '../model/temp'
pathModel = '../model/example/VAE'
patience = 3

# Initialize and train
vae = VAE(dimInput, layerDense=layerDense, ratRecon=ratRecon)
history, timeTrain = vae.fit(xTrain, xTest, 
                             numEpochs=numEpochs,
                             sizeBatch=sizeBatch,
                             nameOptim=nameOptim,
                             pathTempBest=pathTempBest)

# Get the encoder and decoder from the trained model directly
encoder = vae.encoder
decoder = vae.decoder
autoencoder = vae.autoencoder
encoder.summary()
decoder.summary()
autoencoder.summary()

# Save the trained model and load from the path to resume as encoder, decoder, and autoencoder
vae.save(pathModel)
encoder, decoder, autoencoder = VAE.load(pathModel)
encoder.summary()
decoder.summary()
autoencoder.summary()

# Encode testing dataset and get generated reconstruction
encode = encoder.predict(xTest)
generate = decoder.predict(np.array([[0, 0]]))
plt.imshow(generate.reshape(sizeDigit))

# Example of Convolutional VAE

In [ ]:
# Load dataset
(xTrain, yTrain), (xTest, yTest) = mnist.load_data()
xTrain = xTrain.astype('float32') / 255.
xTest = xTest.astype('float32') / 255.
numTrain = len(xTrain)
numTest = len(xTest)
sizeDigit = xTrain.shape[1:]
dimInput = [*xTrain.shape[1:], 1] # dimInput is (width, height, channels)
xTrain = xTrain.reshape((numTrain, *dimInput))
xTest = xTest.reshape((numTest, *dimInput))
print(xTrain.shape)
print(xTest.shape)

# Set parameters
numEpochs = 10
sizeBatch = 128
sizeKernel = 3
layerDense = [16, 2]
layerConv = [4, 16]
ratRecon = 1
nameOptim = 'adam'
pathTempBest = '../model/temp'
pathModel = '../model/example/ConVAE'
patience = 3

# Initialize and train
convVAE = ConvVAE(dimInput, layerDense=layerDense, layerConv=layerConv, ratRecon=ratRecon)
history, timeTrain = convVAE.fit(xTrain, xTest, 
                                 numEpochs=numEpochs,
                                 sizeBatch=sizeBatch,
                                 pathTempBest=pathTempBest)

# Get the encoder and decoder from the trained model directly
encoder = convVAE.encoder
decoder = convVAE.decoder
autoencoder = convVAE.autoencoder
encoder.summary()
decoder.summary()
autoencoder.summary()

# Save the trained model and load from the path to resume as encoder, decoder, and autoencoder
convVAE.save(pathModel)
encoder, decoder, autoencoder = ConvVAE.load(pathModel)
encoder.summary()
decoder.summary()
autoencoder.summary()

# Encode testing dataset and get generated reconstruction
encode = encoder.predict(xTest)
generate = decoder.predict(np.array([[0, 0]]))
plt.imshow(generate.reshape(sizeDigit))